# スケーリング意味なし

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np
import math
import validate
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import PR_curve
import extract_sound

In [26]:
data_dir='fixed_csv_label'
df = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
df=df[df['day']<=9]
df

,original,label,time,day
0,53.4,0,15,0
1,52.8,0,15,0
2,52.8,0,15,0
3,53.2,0,15,0
4,53.6,0,15,0
...,...,...,...,...
3995345,45.9,0,5,9
3995346,45.6,0,5,9
3995347,45.8,0,5,9
3995348,45.7,0,5,9


In [27]:
def make_data(x, str,w):
    x=pd.DataFrame(x)
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)
    return x

def make_label(x, w):
    window=w
    x['label_all']=x['label']
    for i in range(1, window):
        x.loc[(x['label_all']==1)|(x['label'].shift(i)==1), 'label_all']=1
        x.loc[(x['label_all']==0)&(x['label'].shift(i)==0), 'label_all']=0
    
    x['label']=x['label_all']
    return x[['label']]

df_list=[]
for i in set(df['day']):
    lag=[] 
    window=5
    lag.append(make_data(df.loc[df['day']==i, ['original']], 'original', window))
    lag.append(df.loc[df['day']==i, ['day']])
    lag.append(make_label(df.loc[df['day']==i, ['label']], window))
    df_add_lag=pd.concat(lag, axis=1)
    df_list.append(df_add_lag)
df=pd.concat(df_list)
df=df.dropna().reset_index(drop=True)
df

,original,original_lag1,original_lag2,original_lag3,original_lag4,day,label
0,53.6,53.2,52.8,52.8,53.4,0,0
1,54.0,53.6,53.2,52.8,52.8,0,0
2,54.7,54.0,53.6,53.2,52.8,0,0
3,55.7,54.7,54.0,53.6,53.2,0,0
4,55.5,55.7,54.7,54.0,53.6,0,0
...,...,...,...,...,...,...,...
3995305,45.9,45.6,45.4,45.9,46.0,9,0
3995306,45.6,45.9,45.6,45.4,45.9,9,0
3995307,45.8,45.6,45.9,45.6,45.4,9,0
3995308,45.7,45.8,45.6,45.9,45.6,9,0


In [34]:
test_day=[2]
train=df[(df['label']==0)&(~df['day'].isin(test_day))][df.columns[:-2]].reset_index(drop=True)
test=df[(df['day'].isin(test_day))][df.columns[:-2]].reset_index(drop=True)
test

,original,original_lag1,original_lag2,original_lag3,original_lag4
0,45.5,45.2,45.4,45.6,45.4
1,46.0,45.5,45.2,45.4,45.6
2,45.7,46.0,45.5,45.2,45.4
3,45.5,45.7,46.0,45.5,45.2
4,45.7,45.5,45.7,46.0,45.5
...,...,...,...,...,...
431976,53.6,54.3,55.1,55.2,55.6
431977,53.8,53.6,54.3,55.1,55.2
431978,53.1,53.8,53.6,54.3,55.1
431979,53.0,53.1,53.8,53.6,54.3


In [23]:
#標準化
tr_scaler = StandardScaler()
tr_scaler.fit(train)
train = tr_scaler.transform(train)

te_scaler = StandardScaler()
te_scaler.fit(test)
test = te_scaler.transform(test)
test

array([[-0.99118829, -1.05363103, -1.02241037, -1.03802106, -1.05363175],
       [-0.97557769, -0.9911886 , -1.05363159, -1.02241045, -1.03802114],
       [-1.0380201 , -0.975578  , -0.99118915, -1.05363167, -1.02241053],
       ...,
       [-1.0536307 , -1.06924163, -1.11607403, -1.1004635 , -1.17851664],
       [-1.13168372, -1.05363103, -1.0692422 , -1.11607411, -1.10046358],
       [-1.11607312, -1.13168406, -1.05363159, -1.06924228, -1.1160742 ]])

In [35]:
#正規化
tr_mm=MinMaxScaler()
train=tr_mm.fit_transform(train)

te_mm=MinMaxScaler()
test=te_mm.fit_transform(test)
test

array([[0.02892562, 0.02272727, 0.0268595 , 0.03099174, 0.0268595 ],
       [0.0392562 , 0.02892562, 0.02272727, 0.0268595 , 0.03099174],
       [0.03305785, 0.0392562 , 0.02892562, 0.02272727, 0.0268595 ],
       ...,
       [0.18595041, 0.20041322, 0.19628099, 0.2107438 , 0.22727273],
       [0.1838843 , 0.18595041, 0.20041322, 0.19628099, 0.2107438 ],
       [0.15495868, 0.1838843 , 0.18595041, 0.20041322, 0.19628099]])

In [36]:
nk = 10
neigh = NearestNeighbors(n_neighbors=nk)
neigh.fit(train)
d = neigh.kneighbors(test)[0]

d = np.mean(d, axis=1)
mx = np.max(d)
d = d / mx

In [37]:
# 評価用のデータを改めて用意し、異常度を割り当てる
original = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
test_v=original[original['day'].isin(test_day)][['original', 'label']].reset_index(drop=True)
test_v['d']=d[0]
test_v.loc[window-1:, 'd']=d
d=test_v['d'].values

## 独自評価関数

In [38]:
thr_df=pd.read_csv('day_thr.csv')
thr_df

,day,thr
0,0,0.105263
1,1,0.157895
2,2,0.105263
3,3,0.157895
4,4,0.105263
5,5,0.157895
6,6,0.157895
7,7,0.052632
8,8,0.052632
9,9,0.105263


In [39]:
thr=float(thr_df.loc[thr_df['day'].isin(test_day), 'thr'])
precision, recall=validate.validate(test_v, d, thr)
print(f'適合率：{precision}')
print(f'再現率：{recall}')

適合率：0.5883054892601431
再現率：0.53125


## 異常音を抜き出す

In [ ]:
extract_sound.extract_sound(test_v[['z']], test_day, window)

## グラフ描画

In [ ]:
anorm=(d*5)**2
validate.figure(test_v, anorm, d, thr)